In [ ]:
import hist
import keras_tuner as kt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import tensorflow.keras.backend as K
from iminuit import Minuit, cost
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import (
    Conv1D,
    Dense,
    Dropout,
    Flatten,
    Input,
    MaxPooling1D,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

import plotting
from CBAM import CBAM

In [ ]:
plt.style.use(["science", "notebook"])

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["axes.formatter.limits"] = -5, 4
plt.rcParams["figure.figsize"] = 6, 4
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
data_path = "/eos/experiment/sndlhc/users/olantwin/advsnd/2024/07/nu12/CCDIS/"

In [ ]:
df = pd.read_csv(data_path + "features_CNN_1d_99987.csv")

In [ ]:
X = np.load(data_path + "images_1d_99987.npy")[:, :, np.newaxis]

In [ ]:
X /= X.max()

In [ ]:
K.set_image_data_format("channels_last")

In [ ]:
y = df["start_z"].values

In [ ]:
y = (y + 235) / 155

In [ ]:
def y_to_z(y):
    return (y * 155) - 235

In [ ]:
y_to_z(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
num_samples, num_timesteps, num_features = X_train.shape
X_train_reshaped = X_train.reshape(-1, num_features)
X_test_reshaped = X_test.reshape(-1, num_features)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

X_train = X_train_scaled.reshape(num_samples, num_timesteps, num_features)
X_test = X_test_scaled.reshape(X_test.shape[0], X_test.shape[1], num_features)

In [ ]:
X_training, X_validation, y_training, y_validation = train_test_split(
    X_train, y_train, random_state=0
)

In [ ]:
print(X_training.shape, X_validation.shape, y_training.shape, y_validation.shape)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
input_shape = (200, 1)

In [ ]:
model_name = "CNN_1d_finsteraarhorn"

In [ ]:
def build_model(hp):
    model = Sequential()

    model.add(Input(input_shape, name="Input"))

    model.add(
        Conv1D(
            filters=hp.Int("filters_1", min_value=16, max_value=64, step=16),
            kernel_size=hp.Int("kernel_size_1", min_value=2, max_value=6, step=1),
            padding="same",
            activation="elu",
            name="Conv1D_1",
        )
    )
    model.add(CBAM(name="CBAM_1"))

    model.add(
        Conv1D(
            filters=hp.Int("filters_2", min_value=16, max_value=64, step=16),
            kernel_size=hp.Int("kernel_size_2", min_value=2, max_value=6, step=1),
            padding="same",
            activation="elu",
            name="Conv1D_2",
        )
    )
    model.add(CBAM(name="CBAM_2"))

    model.add(MaxPooling1D(2, name="MaxPool1D_1"))

    model.add(
        Conv1D(
            filters=hp.Int("filters_3", min_value=16, max_value=64, step=16),
            kernel_size=hp.Int("kernel_size_3", min_value=2, max_value=6, step=1),
            padding="same",
            activation="elu",
            name="Conv1D_3",
        )
    )
    model.add(CBAM(name="CBAM_3"))

    model.add(MaxPooling1D(3, name="MaxPool1D_2"))

    model.add(
        Dropout(hp.Float(min_value=0.2, max_value=0.6, step=0.1, name="Dropout_1"))
    )

    model.add(
        Conv1D(
            filters=hp.Int("filters_4", min_value=16, max_value=64, step=16),
            kernel_size=hp.Int("kernel_size_4", min_value=2, max_value=6, step=1),
            padding="same",
            activation="elu",
            name="Conv1D_4",
        )
    )
    model.add(CBAM(name="CBAM_4"))

    model.add(Flatten(name="Flat_1"))

    model.add(Dense(1, name="Dense_1"))

    model.compile(
        optimizer=Adam(
            learning_rate=hp.Float(
                "learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"
            )
        ),
        loss="mse",
        metrics=["mae"],
    )

    return model

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor="loss", factor=0.5, patience=6, min_lr=1e-6, verbose=1
)

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective="val_mae",
    max_epochs=16,
    factor=3,
    directory="1D_hyperparam_opt",
    project_name=model_name,
)

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(
    X_training,
    y_training,
    validation_data=(X_validation, y_validation),
    callbacks=[reduce_lr],
)

In [ ]:
tuner.results_summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
history_df = None

In [ ]:
fit_result = model.fit(x=X_train, y=y_train, batch_size=128, epochs=100)

In [ ]:
history_df = pd.concat([history_df, pd.DataFrame(fit_result.history)])

In [ ]:
model.save(f"{model_name}_n{len(y)}_e{len(history_df)}.keras")

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
plt.title("CNN start z")
ax1.plot(history_df["loss"].values, color=colors[0])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Loss Function", color=colors[0])

ax1.set_ylim([0, 0.025])

ax2.plot(history_df["mae"].values, color=colors[1])
ax2.set_ylabel("Error", color=colors[1])

ax2.set_ylim([0, 0.13])

plt.text(
    0.3,
    0.7,
    f"Training dataset: {len(y_train)} events\n"
    f"Test dataset: {len(y_test)} events\n"
    f"Training duration: {len(history_df)} epochs\n{model_name}",
    transform=ax1.transAxes,
    fontsize=12,
)
plt.savefig(f"plots/1d_cnn/convergence_{model_name}_n{len(y)}_e{len(history_df)}.pdf")
plt.savefig(
    f"plots/1d_cnn/convergence_{model_name}_n{len(y)}_e{len(history_df)}.png",
    facecolor="white",
)

In [ ]:
y_pred = model.predict(x=X_test)

In [ ]:
z_pred = y_to_z(y_pred)

In [ ]:
h = hist.Hist.new.Regular(20, -5, +5, name=r"𝛥z [cm]").Double()

In [ ]:
h.fill(np.squeeze(z_pred) - np.squeeze(y_to_z(y_test)))

In [ ]:
entries, edges = h.to_numpy()

In [ ]:
def residual_model(x, mu, sigma):
    return scipy.stats.norm.cdf(x, mu, sigma)

In [ ]:
m = Minuit(cost.BinnedNLL(entries, edges, residual_model), 0, 25)

In [ ]:
res = m.migrad()

In [ ]:
res

In [ ]:
h.plot()
plt.xlabel(r"$\Delta z\;[\mathrm{cm}]$")
plt.title("Improved CNN")
plot_range = edges[0], edges[-1]
x = np.linspace(*plot_range, 100)
best_fit = scipy.stats.norm(res.params[0].value, res.params[1].value)
n_bins = len(entries)
binsize = (plot_range[1] - plot_range[0]) / n_bins
scale = h.sum() / (best_fit.cdf(plot_range[1]) - best_fit.cdf(plot_range[0])) * binsize
plt.plot(x, scale * best_fit.pdf(x))

plt.ylim([0, 8000])

ax = plt.gca()
plt.text(
    0.6,
    0.9,
    rf"$\mu = {res.params[0].value:.2f} \pm {res.params[0].error:.3f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
    fontsize=11,
)
plt.text(
    0.025,
    0.72,
    f"Training dataset: {len(y_train)} events\n"
    f"Test dataset: {len(y_test)} events\n"
    f"Training duration: {len(history_df)} epochs\n{model_name}",
    transform=ax.transAxes,
    usetex=True,
    fontsize=11,
)
plt.text(
    0.6,
    0.81,
    rf"$\sigma = {res.params[1].value:.2f} \pm {res.params[1].error:.3f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
    fontsize=12,
)
plotting.watermark()
plt.savefig(f"plots/1d_cnn/h_dz_{model_name}_n{len(y)}_e{len(history_df)}.pdf")
plt.savefig(
    f"plots/1d_cnn/h_dz_{model_name}_n{len(y)}_e{len(history_df)}.png",
    facecolor="white",
)